In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pdb
import scipy.stats as stats
import scipy
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests


In [4]:
data_dir = f'/lab_data/behrmannlab/claire/pepdoc/results_ex1' #read in the file; first value is the file name
curr_dir = f'/user_data/vayzenbe/GitHub_Repos/pepdoc' #CHANGE AS NEEEDED CAUSE ITS FOR VLAAAD
bin_size = 5 #20 ms bins (EACH BIN IS 4 MS SO 5 ROWS ARE 20 MS)
# bin_size = 1 
categories = ['tool','nontool','bird','insect']
labels = np.asanyarray([0]*5 + [1]*5 + [2]*5 + [3]*5) #creates labels for data

#d_channels
channels = [77, 78, 79, 80, 86, 87, 88, 89, 98, 99, 100, 110, 109, 118, 131, 143, 154, 163, 130, 142, 153, 162, 129, 141, 152, 128, 140, 127] # a list of channels
columns  =[f'E{ii}' for ii in channels] #convert channels into the same format as the columns

#v_channels
ventral_channels = [104, 105, 106, 111, 112, 113, 114, 115, 120, 121, 122, 123, 133, 134, 169, 177, 189, 159, 168, 176, 18, 199, 158, 167, 175, 187, 166, 174] # a list of channels
ventral_columns  =[f'E{ii}' for ii in ventral_channels] #convert channels into the same format as the columns

#c_channels
control_channels =  [11, 12, 18, 19, 20, 21, 25, 26, 27, 32, 33, 34, 37, 38]
control_columns  =[f'E{ii}' for ii in control_channels] #convert channels into the same format as the columns

svm_test_size = .4
svm_splits = 20
sss = StratifiedShuffleSplit(n_splits=svm_splits, test_size=svm_test_size)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [6]:
d_sub_decode =np.load(f'{curr_dir}/results/d_sub_decode.npy')
v_sub_decode =np.load(f'{curr_dir}/results/v_sub_decode.npy')
c_sub_decode =np.load(f'{curr_dir}/results/c_sub_decode.npy')

In [7]:
'''Time Comparison'''
#dorsal
d_sub_decode = np.asanyarray(d_sub_decode)
d_timecomp = []
for i in range(0, d_sub_decode.shape[1]):
    a = stats.ttest_1samp((d_sub_decode[:,i]), .25, axis = 0, alternative='greater')
    d_timecomp.append(a)


#ventral
v_sub_decode = np.asanyarray(v_sub_decode)
v_timecomp = []
for i in range(0, v_sub_decode.shape[1]):
    a = stats.ttest_1samp((v_sub_decode[:,i]), .25, axis = 0, alternative='greater')
    v_timecomp.append(a)


#control
c_sub_decode = np.asanyarray(c_sub_decode)
c_timecomp = []
for i in range(0, c_sub_decode.shape[1]):
    a = stats.ttest_1samp((c_sub_decode[:,i]), .25, axis = 0, alternative='greater')
    c_timecomp.append(a)



In [8]:
#RESAMPLING

#how many times to do the resampling
#we'll eventually set this to somethign really high like 10,000
iter = 100

d_sub_decode_short = d_sub_decode[:, 9:]
v_sub_decode_short = v_sub_decode[:, 9:]
c_sub_decode_short = c_sub_decode[:, 9:]
#subs = np.arange(0,20).reshape(20,1)
#d_sub_decode_short = np.append(subs, d_sub_decode,1)
#v_sub_decode_short = np.append(subs, v_sub_decode,1)
#c_sub_decode_short = np.append(subs, c_sub_decode,1)
#d_sub_decode_short = d_sub_decode

#d_sub_decode_short = d_sub_decode

#convert data to pandas dataframe
#this is just because pandas has a good resampling function 
d_sub_decode_short = pd.DataFrame(d_sub_decode_short)
v_sub_decode_short = pd.DataFrame(v_sub_decode_short)
c_sub_decode_short = pd.DataFrame(c_sub_decode_short)

#Create empty lists that will hold the results of each resample
d_boot = []
v_boot = []
c_boot = []

d_sub_counts = np.zeros((1,134))
v_sub_counts = np.zeros((1,134))
c_sub_counts = np.zeros((1,134))

for ii in range(0,iter):
    
    #resample the sub decode data with replacement
    d_sub_sample = d_sub_decode_short.sample(d_sub_decode_short.shape[0],replace = True, random_state=ii)
    v_sub_sample = v_sub_decode_short.sample(v_sub_decode_short.shape[0],replace = True, random_state=ii)
    c_sub_sample = c_sub_decode_short.sample(c_sub_decode_short.shape[0],replace = True, random_state=ii)
    
    #convert it back to a numpy array
    d_sub_sample = d_sub_sample.to_numpy() 
    v_sub_sample = v_sub_sample.to_numpy()
    c_sub_sample = c_sub_sample.to_numpy()

    #calculate the bootstrap sample mean
    d_timecomp = []
    v_timecomp = []
    c_timecomp = []
    
    for time in range(0,d_sub_sample.shape[1]):
        d_stat= stats.ttest_1samp(d_sub_sample[:,time], .25, axis = 0, alternative='greater')
        v_stat = stats.ttest_1samp(v_sub_sample[:,time], .25, axis = 0, alternative='greater')
        c_stat = stats.ttest_1samp(c_sub_sample[:,time], .25, axis = 0, alternative='greater')

        #append the p-value for every time point
        d_timecomp.append(d_stat[1])  
        v_timecomp.append(v_stat[1])
        c_timecomp.append(c_stat[1])
    
    #reconvert p-value list into a numpy array
    d_timecomp = np.asanyarray(d_timecomp)
    v_timecomp = np.asanyarray(v_timecomp)
    c_timecomp = np.asanyarray(c_timecomp)

    
    #find the the first time point that is below change (0.05)
    #np.where simply returns the indices (i.e., spots in an array), that meet some condition
    #i'm simply grabbing the first value of that list, which corresponds to the first time point above chance
    d_onset = np.where(d_timecomp <.05,)[0][0]
    v_onset = np.where(v_timecomp <.05,)[0][0]
    c_onset = np.where(c_timecomp <.05,)[0][0]

    d_sub_counts[0,np.where(d_timecomp <.05)[0]] += 1
    v_sub_counts[0,np.where(v_timecomp <.05)[0]] += 1
    c_sub_counts[0,np.where(c_timecomp <.05)[0]] += 1

    #if d_onset == 1:
    #    pdb.set_trace()
    
    #convert to the actual time point
    d_onset_converted = d_onset
    v_onset_converted = v_onset
    c_onset_converted = c_onset

    #d_onset_converted = d_onset *4
    #v_onset_converted = v_onset *4
    #c_onset_converted = c_onset *4

    #add the onset value from the resample to a list
    d_boot.append(d_onset_converted)
    v_boot.append(v_onset_converted)
    c_boot.append(c_onset_converted)